In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')
#path = '/content/gdrive/My Drive/Colab Notebooks/707/Labs/'
#import sys
#sys.path.append(path)

In [ ]:
!pip install Box2D
!pip install box2d-py
!pip install gym[all]
!pip install gym[Box_2D]
!pip install torc
!pip install -U "ray[rllib]" torch
import gym 
env = gym.make("LunarLander-v2")

  Using cached mujoco-py-1.50.1.68.tar.gz (120 kB)
  ERROR: Failed building wheel for mujoco-py
  Running setup.py clean for mujoco-py
Failed to build mujoco-py
    Running setup.py install for mujoco-py ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-nbj3qvf7/mujoco-py_8397aa03472a45909340414fb455142b/setup.py'"'"'; __file__='"'"'/tmp/pip-install-nbj3qvf7/mujoco-py_8397aa03472a45909340414fb455142b/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-_v5a1a3m/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.7/mujoco-py Check the logs for full command output.
     |████

In [19]:
import ray
import ray.rllib.agents.dqn as dqn

def evaluation_fn(result):
    return result['episode_reward_mean']


def objective_fn(config):

    
    trainer = dqn.DQNTrainer(config=config)

    for i in range(10):
      # Perform one iteration of training the policy with DQN
      result = trainer.train()
      intermediate_score = evaluation_fn(result)
      
      # Feed the score back back to Tune.
      tune.report(iterations=i, mean_reward=intermediate_score)
      

## Use DQN and train your algorithm on the Dungeon environment.

You can take inspiration from:
https://docs.ray.io/en/latest/rllib/rllib-training.html#basic-python-api

Experiment with the different parameters of the configuration:
https://docs.ray.io/en/latest/rllib/rllib-algorithms.html#deep-q-networks-dqn-rainbow-parametric-dqn





In [21]:

import ray
import ray.rllib.agents.dqn as dqn
from ray.tune.logger import pretty_print
from ray import tune 

config = dqn.DEFAULT_CONFIG.copy()
config["dueling"] = tune.grid_search([True, False])
config["double_q"] = tune.grid_search([True, False])
config["model"] = { "fcnet_hiddens": [128, 128, 128],
                    "fcnet_activation": 'relu',
    }
config["env"] = "LunarLander-v2"
#config['lr'] = tune.loguniform(1e-4, 1e-1),
config["gamma"] = tune.uniform(0, 1)

analysis = tune.run(
        objective_fn,
        metric="mean_reward",
        mode="max",
        num_samples=3,
        name='HP_tuning_LunarLander',
        config=config)


print("Best hyperparameters found were: ", analysis.best_config)


Trial name,status,loc,double_q,dueling,gamma
objective_fn_LunarLander-v2_ec25e_00000,PENDING,,True,True,0.920612
objective_fn_LunarLander-v2_ec25e_00001,PENDING,,False,True,0.0806447
objective_fn_LunarLander-v2_ec25e_00002,PENDING,,True,False,0.263824
objective_fn_LunarLander-v2_ec25e_00003,PENDING,,False,False,0.15326
objective_fn_LunarLander-v2_ec25e_00004,PENDING,,True,True,0.0575363
objective_fn_LunarLander-v2_ec25e_00005,PENDING,,False,True,0.835263
objective_fn_LunarLander-v2_ec25e_00006,PENDING,,True,False,0.656405
objective_fn_LunarLander-v2_ec25e_00007,PENDING,,False,False,0.867242
objective_fn_LunarLander-v2_ec25e_00008,PENDING,,True,True,0.908645
objective_fn_LunarLander-v2_ec25e_00009,PENDING,,False,True,0.592168


(ImplicitFunc pid=5396) 2022-04-13 12:55:02,868	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(objective_fn pid=5396) 2022-04-13 12:55:02,889	INFO trainer.py:2141 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(objective_fn pid=5396) 2022-04-13 12:55:02,890	INFO simple_q.py:155 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
(objective_fn pid=5396) 2022-04-13 12:55:02,890	INFO trainer.py:781 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(ImplicitFunc pid=5398) 2022-04-13 12:55:02,882	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(objective_fn pid=5398) 2022-04-13 12:55

Trial name,status,loc,double_q,dueling,gamma
objective_fn_LunarLander-v2_ec25e_00000,RUNNING,172.28.0.2:5401,True,True,0.920612
objective_fn_LunarLander-v2_ec25e_00001,RUNNING,172.28.0.2:5393,False,True,0.0806447
objective_fn_LunarLander-v2_ec25e_00002,RUNNING,172.28.0.2:5396,True,False,0.263824
objective_fn_LunarLander-v2_ec25e_00003,RUNNING,172.28.0.2:5398,False,False,0.15326
objective_fn_LunarLander-v2_ec25e_00004,RUNNING,172.28.0.2:5399,True,True,0.0575363
objective_fn_LunarLander-v2_ec25e_00005,RUNNING,172.28.0.2:5400,False,True,0.835263
objective_fn_LunarLander-v2_ec25e_00006,RUNNING,172.28.0.2:5397,True,False,0.656405
objective_fn_LunarLander-v2_ec25e_00007,RUNNING,172.28.0.2:5402,False,False,0.867242
objective_fn_LunarLander-v2_ec25e_00008,RUNNING,172.28.0.2:5395,True,True,0.908645
objective_fn_LunarLander-v2_ec25e_00009,RUNNING,172.28.0.2:5391,False,True,0.592168


(objective_fn pid=5396) 2022-04-13 12:55:04,058	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(objective_fn pid=5398) 2022-04-13 12:55:04,066	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(objective_fn pid=5397) 2022-04-13 12:55:04,125	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(objective_fn pid=5402) 2022-04-13 12:55:04,140	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(objective_fn pid=5394) 2022-04-13 12:55:04,128	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(objective_fn pid=5390) 2022-04-13 12:55:04,139	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(objective_fn pid=5399) 2022-04-13 12:55:04,419	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(objective_fn pid=5400) 2022-04-13 12:55:04,419	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(objective_fn pid=5393) 2022-04-13 12:55:04,377	WARNING util.py:55 -- Install gputil for GPU system moni

Result for objective_fn_LunarLander-v2_ec25e_00002:
  date: 2022-04-13_12-55-06
  done: false
  experiment_id: 72e9c57841e847a189aac6fa2efac696
  hostname: aa9ef03c8f27
  iterations: 0
  iterations_since_restore: 1
  mean_reward: -192.4890648134936
  node_ip: 172.28.0.2
  perf:
    cpu_util_percent: 34.86
    ram_util_percent: 18.4
  pid: 5396
  time_since_restore: 3.3256022930145264
  time_this_iter_s: 3.3256022930145264
  time_total_s: 3.3256022930145264
  timestamp: 1649854506
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: ec25e_00002
  
Result for objective_fn_LunarLander-v2_ec25e_00003:
  date: 2022-04-13_12-55-06
  done: false
  experiment_id: 0f6a6d8406164452905142864f7bd02d
  hostname: aa9ef03c8f27
  iterations: 0
  iterations_since_restore: 1
  mean_reward: -152.74255867612288
  node_ip: 172.28.0.2
  perf:
    cpu_util_percent: 34.88
    ram_util_percent: 18.42
  pid: 5398
  time_since_restore: 3.320237636566162
  time_this_iter_s: 3.320237636566162
  time_to

...

Trial name,status,loc,double_q,dueling,gamma,iter,total time (s),iterations,mean_reward
objective_fn_LunarLander-v2_ec25e_00000,RUNNING,172.28.0.2:5401,True,True,0.920612,9,42.8793,8,-118.816
objective_fn_LunarLander-v2_ec25e_00005,RUNNING,172.28.0.2:5400,False,True,0.835263,9,42.2712,8,-153.346
objective_fn_LunarLander-v2_ec25e_00007,RUNNING,172.28.0.2:5402,False,False,0.867242,9,42.2357,8,-128.431
objective_fn_LunarLander-v2_ec25e_00008,RUNNING,172.28.0.2:5395,True,True,0.908645,9,43.6904,8,-152.922
objective_fn_LunarLander-v2_ec25e_00009,RUNNING,172.28.0.2:5391,False,True,0.592168,10,44.6361,9,-145.417
objective_fn_LunarLander-v2_ec25e_00010,RUNNING,172.28.0.2:5394,True,False,0.96893,9,40.3708,8,-144.197
objective_fn_LunarLander-v2_ec25e_00011,RUNNING,172.28.0.2:5390,False,False,0.314437,9,40.1896,8,-137.388
objective_fn_LunarLander-v2_ec25e_00001,TERMINATED,172.28.0.2:5393,False,True,0.0806447,10,43.6335,9,-157.901
objective_fn_LunarLander-v2_ec25e_00002,TERMINATED,172.28.0.2:5396,True,False,0.263824,10,42.0132,9,-142.206
objective_fn_LunarLander-v2_ec25e_00003,TERMINATED,172.28.0.2:5398,False,False,0.15326,10,43.0801,9,-157.547


Result for objective_fn_LunarLander-v2_ec25e_00009:
  date: 2022-04-13_12-55-47
  done: true
  experiment_id: ffe6dbf0660146ddac1f430fa95cce4e
  experiment_tag: 9_double_q=False,dueling=True,gamma=0.59217
  hostname: aa9ef03c8f27
  iterations: 9
  iterations_since_restore: 10
  mean_reward: -145.41683203308696
  node_ip: 172.28.0.2
  perf:
    cpu_util_percent: 35.6
    ram_util_percent: 18.185714285714287
  pid: 5391
  time_since_restore: 44.63611960411072
  time_this_iter_s: 4.612032651901245
  time_total_s: 44.63611960411072
  timestamp: 1649854547
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: ec25e_00009
  
Result for objective_fn_LunarLander-v2_ec25e_00011:
  date: 2022-04-13_12-55-47
  done: true
  experiment_id: 9cc718012e4e4ba1a698ca9754aa24b8
  experiment_tag: 11_double_q=False,dueling=False,gamma=0.31444
  hostname: aa9ef03c8f27
  iterations: 9
  iterations_since_restore: 10
  mean_reward: -140.1744855914592
  node_ip: 172.28.0.2
  perf:
    cpu_util_perce

Trial name,status,loc,double_q,dueling,gamma,iter,total time (s),iterations,mean_reward
objective_fn_LunarLander-v2_ec25e_00000,TERMINATED,172.28.0.2:5401,True,True,0.920612,10,48.1191,9,-120.595
objective_fn_LunarLander-v2_ec25e_00001,TERMINATED,172.28.0.2:5393,False,True,0.0806447,10,43.6335,9,-157.901
objective_fn_LunarLander-v2_ec25e_00002,TERMINATED,172.28.0.2:5396,True,False,0.263824,10,42.0132,9,-142.206
objective_fn_LunarLander-v2_ec25e_00003,TERMINATED,172.28.0.2:5398,False,False,0.15326,10,43.0801,9,-157.547
objective_fn_LunarLander-v2_ec25e_00004,TERMINATED,172.28.0.2:5399,True,True,0.0575363,10,43.2535,9,-157.788
objective_fn_LunarLander-v2_ec25e_00005,TERMINATED,172.28.0.2:5400,False,True,0.835263,10,47.0095,9,-161.419
objective_fn_LunarLander-v2_ec25e_00006,TERMINATED,172.28.0.2:5397,True,False,0.656405,10,43.2442,9,-154.597
objective_fn_LunarLander-v2_ec25e_00007,TERMINATED,172.28.0.2:5402,False,False,0.867242,10,46.8652,9,-130.107
objective_fn_LunarLander-v2_ec25e_00008,TERMINATED,172.28.0.2:5395,True,True,0.908645,10,48.483,9,-151.812
objective_fn_LunarLander-v2_ec25e_00009,TERMINATED,172.28.0.2:5391,False,True,0.592168,10,44.6361,9,-145.417


2022-04-13 12:55:51,590	INFO tune.py:639 -- Total run time: 53.93 seconds (53.74 seconds for the tuning loop).


Best hyperparameters found were:  {'num_workers': 0, 'num_envs_per_worker': 1, 'create_env_on_driver': False, 'rollout_fragment_length': 4, 'batch_mode': 'truncate_episodes', 'gamma': 0.9206117943788054, 'lr': 0.0005, 'train_batch_size': 32, 'model': {'fcnet_hiddens': [128, 128, 128], 'fcnet_activation': 'relu'}, 'optimizer': {}, 'horizon': None, 'soft_horizon': False, 'no_done_at_end': False, 'env': 'LunarLander-v2', 'observation_space': None, 'action_space': None, 'env_config': {}, 'remote_worker_envs': False, 'remote_env_batch_wait_ms': 0, 'env_task_fn': None, 'render_env': False, 'record_env': False, 'clip_rewards': None, 'normalize_actions': True, 'clip_actions': False, 'preprocessor_pref': 'deepmind', 'log_level': 'WARN', 'callbacks': <class 'ray.rllib.agents.callbacks.DefaultCallbacks'>, 'ignore_worker_failures': False, 'log_sys_usage': True, 'fake_sampler': False, 'framework': 'tf', 'eager_tracing': False, 'eager_max_retraces': 20, 'explore': True, 'exploration_config': {'type'

In [22]:
print("Best hyperparameters found were: ", analysis.best_config)

Best hyperparameters found were:  {'num_workers': 0, 'num_envs_per_worker': 1, 'create_env_on_driver': False, 'rollout_fragment_length': 4, 'batch_mode': 'truncate_episodes', 'gamma': 0.9206117943788054, 'lr': 0.0005, 'train_batch_size': 32, 'model': {'fcnet_hiddens': [128, 128, 128], 'fcnet_activation': 'relu'}, 'optimizer': {}, 'horizon': None, 'soft_horizon': False, 'no_done_at_end': False, 'env': 'LunarLander-v2', 'observation_space': None, 'action_space': None, 'env_config': {}, 'remote_worker_envs': False, 'remote_env_batch_wait_ms': 0, 'env_task_fn': None, 'render_env': False, 'record_env': False, 'clip_rewards': None, 'normalize_actions': True, 'clip_actions': False, 'preprocessor_pref': 'deepmind', 'log_level': 'WARN', 'callbacks': <class 'ray.rllib.agents.callbacks.DefaultCallbacks'>, 'ignore_worker_failures': False, 'log_sys_usage': True, 'fake_sampler': False, 'framework': 'tf', 'eager_tracing': False, 'eager_max_retraces': 20, 'explore': True, 'exploration_config': {'type'

In [23]:
df = analysis.dataframe(metric="mean_reward", mode="max")
df[['config/gamma', 'config/double_q', 'config/dueling', 'mean_reward']]

,config/gamma,config/double_q,config/dueling,mean_reward
0,0.920612,True,True,-118.815585
1,0.080645,False,True,-134.369129
2,0.263824,True,False,-139.913083
3,0.153260,False,False,-130.480041
4,0.057536,True,True,-147.929386
5,0.835263,False,True,-143.148969
6,0.656405,True,False,-149.418683
7,0.867242,False,False,-127.519113
8,0.908645,True,True,-151.811692
9,0.592168,False,True,-127.730261


https://docs.ray.io/en/latest/tune/api_docs/analysis.html

In [24]:
from ray.tune import ExperimentAnalysis
analysis_result = ExperimentAnalysis("~/ray_results/HP_tuning_LunarLander")

2022-04-13 12:56:12,105	INFO experiment_analysis.py:696 -- No `self.trials`. Drawing logdirs from checkpoint file. This may result in some information that is out of sync, as checkpointing is periodic.
